In [1]:
from fastai.text import *   # Quick access to NLP functionality

In [2]:
path = untar_data(URLs.IMDB_SAMPLE)
path

PosixPath('/home/ds/DataScience/home/ademyanchuk/.fastai/data/imdb_sample')

In [3]:
path.ls()

[PosixPath('/home/ds/DataScience/home/ademyanchuk/.fastai/data/imdb_sample/texts.csv'),
 PosixPath('/home/ds/DataScience/home/ademyanchuk/.fastai/data/imdb_sample/models')]

In [4]:
df = pd.read_csv(path/'texts.csv', header=None)
df.head()

,0,1,2
0,label,text,is_valid
1,negative,Un-bleeping-believable! Meg Ryan doesn't even ...,False
2,positive,This is a extremely well-made film. The acting...,False
3,negative,Every once in a long while a movie will come a...,False
4,positive,Name just says it all. I watched this movie wi...,False


In [5]:
data_lm = TextLMDataBunch.from_csv(path, 'texts.csv')

In [13]:
learn = language_model_learner(data_lm)

In [6]:
pretrained_fnames = ['lstm_wt103', 'itos_wt103']

In [7]:
learn = language_model_learner(data_lm, pretrained_fnames=pretrained_fnames)

In [8]:
moms = (0.8,0.7)

In [9]:
learn.unfreeze()

In [10]:
learn.fit_one_cycle(4, slice(1e-2), moms=moms)

epoch,train_loss,valid_loss,accuracy
1,4.618361,3.895687,0.292520
2,4.312834,3.740356,0.297549
3,4.061804,3.704552,0.300940
4,3.888322,3.701622,0.300606
